In [2]:
import os, requests, time
from pprint import pprint

# Use your key if you have one; "3" is the public demo key; "123" also works for v1
API_KEY = os.getenv("THESPORTSDB_KEY", "3")
BASE_URL = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}"

def GET(path, params=None, timeout=15):
    url = f"{BASE_URL}/{path.lstrip('/')}"
    r = requests.get(url, params=params or {}, timeout=timeout)
    r.raise_for_status()
    data = r.json()
    # TheSportsDB often returns {"x": None} when empty → normalize to {}
    if isinstance(data, dict) and data and all(v is None for v in data.values()):
        return {}
    return data

print("Using:", BASE_URL)

Using: https://www.thesportsdb.com/api/v1/json/3


## List sports (what your key can actually see)

In [3]:
sports = (GET("all_sports.php").get("sports") or [])
print("Total sports available:", len(sports))
for s in sports:
    print("-", s.get("strSport"))

Total sports available: 1
- Soccer


## List all leagues (optionally filter by sport)

In [5]:
ALL_LEAGUES = (GET("all_leagues.php").get("leagues") or [])
print("Total leagues (all sports):", len(ALL_LEAGUES))

# Filter helper
def leagues_for_sport(sport_name: str):
    s = sport_name.strip().lower()
    return [L for L in ALL_LEAGUES if (L.get("strSport") or "").strip().lower() == s]

# Example: Soccer leagues
SOCCER_LEAGUES = leagues_for_sport("Soccer")
print("Soccer leagues:", len(SOCCER_LEAGUES))
for L in SOCCER_LEAGUES[:34]:
    print("-", L["idLeague"], L["strLeague"])

Total leagues (all sports): 50
Soccer leagues: 34
- 4328 English Premier League
- 4329 English League Championship
- 4330 Scottish Premier League
- 4331 German Bundesliga
- 4332 Italian Serie A
- 4334 French Ligue 1
- 4335 Spanish La Liga
- 4336 Greek Superleague Greece
- 4337 Dutch Eredivisie
- 4338 Belgian Pro League
- 4339 Turkish Super Lig
- 4340 Danish Superliga
- 4344 Portuguese Primeira Liga
- 4346 American Major League Soccer
- 4347 Swedish Allsvenskan
- 4350 Mexican Primera League
- 4351 Brazilian Serie A
- 4354 Ukrainian Premier League
- 4355 Russian Football Premier League
- 4356 Australian A-League
- 4358 Norwegian Eliteserien
- 4359 Chinese Super League
- 4367 _No League
- 4394 Italian Serie B
- 4395 Scottish Championship
- 4396 English League 1
- 4397 English League 2
- 4398 Italian Serie C Girone C
- 4399 German 2. Bundesliga
- 4400 Spanish La Liga 2
- 4401 French Ligue 2
- 4403 Swedish Superettan
- 4404 Brazilian Serie B
- 4406 Argentinian Primera Division


## (Optional) Leagues by sport + country using the search endpoint

In [6]:
def leagues_by_country_and_sport(country: str, sport: str):
    data = GET("search_all_leagues.php", {"c": country, "s": sport}) or {}
    return data.get("countrys") or []

eng_soc = leagues_by_country_and_sport("England", "Soccer")
print("England + Soccer leagues:", len(eng_soc))
for L in eng_soc:
    print("-", L["idLeague"], L["strLeague"])

England + Soccer leagues: 0


## Teams in a league

In [8]:
LEAGUE_ID = "4328"   # English Premier League (example)
teams_resp = GET("lookup_all_teams.php", {"id": LEAGUE_ID})
teams = teams_resp.get("teams") or []
print(f"Teams in league {LEAGUE_ID}:", len(teams))
for t in teams[:24]:
    print("-", t["idTeam"], t["strTeam"])

Teams in league 4328: 24
- 133606 Bolton Wanderers
- 133607 Wigan Athletic
- 133618 Blackpool
- 133620 Doncaster Rovers
- 133630 Barnsley
- 133631 Peterborough United
- 133633 Reading
- 133637 Cardiff City
- 133836 Plymouth Argyle
- 133888 Luton Town
- 133932 Huddersfield Town
- 134189 Bradford City
- 134231 Rotherham United
- 134241 AFC Wimbledon
- 134258 Stockport County
- 134365 Exeter City
- 134367 Leyton Orient
- 134370 Northampton Town
- 134375 Port Vale
- 134376 Burton Albion
- 134378 Stevenage
- 134381 Mansfield Town
- 134382 Wycombe Wanderers
- 135900 Lincoln City


## Team lookup + players (squad)

In [9]:
# Pick a team id (Arsenal is often 133604; adjust if not present)
TEAM_ID = teams[0]["idTeam"] if teams else "133604"

team_detail = (GET("lookupteam.php", {"id": TEAM_ID}).get("teams") or [{}])[0]
print("Team snapshot:")
pprint({k: team_detail.get(k) for k in [
    "idTeam","strTeam","strLeague","strCountry","intFormedYear",
    "strStadium","strWebsite","strTeamBadge"
]})

players = (GET("lookup_all_players.php", {"id": TEAM_ID}).get("player") or [])
print("\nPlayers in team:", len(players))
for p in players[:12]:
    print("-", p.get("idPlayer"), p.get("strPlayer"), "|", p.get("strPosition"))

Team snapshot:
{'idTeam': '133604',
 'intFormedYear': '1892',
 'strCountry': 'England',
 'strLeague': 'English Premier League',
 'strStadium': 'Emirates Stadium',
 'strTeam': 'Arsenal',
 'strTeamBadge': None,
 'strWebsite': 'www.arsenal.com'}

Players in team: 28
- 34163698 Ben White | Right-Back
- 34169884 Bukayo Saka | Right Winger
- 34164499 Christian Nørgaard | Defensive Midfield
- 34194118 Cristhian Mosquera | Centre-Back
- 34148681 David Raya | Goalkeeper
- 34161584 Declan Rice | Defensive Midfield
- 34200626 Ethan Nwaneri | Attacking Midfield
- 34160962 Gabriel Jesus | Centre-Forward
- 34172252 Gabriel Magalhães | Centre-Back
- 34169883 Gabriel Martinelli | Left Wing
- 34193328 Jakub Kiwior | Centre-Back
- 34172505 Jurrien Timber | Centre-Back


## Search team / player by name

In [10]:
def search_teams(name: str):
    return (GET("searchteams.php", {"t": name}).get("teams") or [])

def search_players(name: str):
    return (GET("searchplayers.php", {"p": name}).get("player") or [])

print("Search team 'Arsenal':", [t["idTeam"] for t in search_teams("Arsenal")][:5])
print("Search player 'Saka':", [p["idPlayer"] for p in search_players("Saka")][:5])

Search team 'Arsenal': ['133604']
Search player 'Saka': ['34155082', '34155213', '34168458', '34168973', '34169884']


## Past / Next matches for a league

In [12]:
def league_matches(league_id: str, kind="past", limit=10):
    if kind == "past":
        raw = (GET("eventspastleague.php", {"id": league_id}).get("events") or [])
    else:
        raw = (GET("eventsnextleague.php", {"id": league_id}).get("events") or [])
    return raw[:limit]

past = league_matches(LEAGUE_ID, "past", 10)
upcoming = league_matches(LEAGUE_ID, "next", 10)

print("Past matches:", len(past))
for e in past[:10]:
    print("-", e.get("dateEvent"), e.get("strEvent"), e.get("intHomeScore"), e.get("intAwayScore"), "| id=", e.get("idEvent"))

print("\nUpcoming matches:", len(upcoming))
for e in upcoming[:10]:
    print("-", e.get("dateEvent"), e.get("strEvent"), "| id=", e.get("idEvent"))

Past matches: 10
- 2025-08-20 Bolton Wanderers vs Reading 1 1 | id= 2274670
- 2025-08-19 AFC Wimbledon vs Cardiff City 0 1 | id= 2274681
- 2025-08-19 Wycombe Wanderers vs Exeter City 0 1 | id= 2274680
- 2025-08-19 Stockport County vs Bradford City 1 2 | id= 2274679
- 2025-08-19 Port Vale vs Stevenage 1 2 | id= 2274677
- 2025-08-19 Plymouth Argyle vs Leyton Orient 0 1 | id= 2274676
- 2025-08-19 Peterborough United vs Barnsley 0 1 | id= 2274675
- 2025-08-19 Northampton Town vs Lincoln City 0 1 | id= 2274674
- 2025-08-19 Mansfield Town vs Blackpool 2 0 | id= 2274673
- 2025-08-19 Luton Town vs Wigan Athletic 1 0 | id= 2274672

Upcoming matches: 10
- 2025-08-23 AFC Wimbledon vs Barnsley | id= 2274682
- 2025-08-23 Bolton Wanderers vs Lincoln City | id= 2274683
- 2025-08-23 Huddersfield Town vs Stevenage | id= 2274684
- 2025-08-23 Mansfield Town vs Leyton Orient | id= 2274685
- 2025-08-23 Northampton Town vs Exeter City | id= 2274686
- 2025-08-23 Peterborough United vs Bradford City | id= 227

## Seasons for a league + full schedule for a season

In [13]:
seasons = [s["strSeason"] for s in (GET("search_all_seasons.php", {"id": LEAGUE_ID}).get("seasons") or [])]
print("Seasons:", seasons[:10])

if seasons:
    SZN = seasons[-1]  # pick the latest available
    season_events = (GET("eventsseason.php", {"id": LEAGUE_ID, "s": SZN}).get("events") or [])
    print(f"\nSeason {SZN} events:", len(season_events))
    for e in season_events[:8]:
        print("-", e.get("dateEvent"), e.get("strEvent"), "| id=", e.get("idEvent"))

Seasons: ['1992-1993', '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998', '1998-1999', '1999-2000', '2000-2001', '2001-2002']

Season 2025-2026 events: 100
- 2025-08-15 Liverpool vs Bournemouth | id= 2267073
- 2025-08-16 Aston Villa vs Newcastle United | id= 2267074
- 2025-08-16 Brighton and Hove Albion vs Fulham | id= 2267075
- 2025-08-16 Sunderland vs West Ham United | id= 2267077
- 2025-08-16 Tottenham Hotspur vs Burnley | id= 2267078
- 2025-08-16 Wolverhampton Wanderers vs Manchester City | id= 2267079
- 2025-08-17 Nottingham Forest vs Brentford | id= 2267076
- 2025-08-17 Chelsea vs Crystal Palace | id= 2267080


## Matches by day (all leagues) for a sport

In [14]:
DAY = "2025-08-17"
day_events = (GET("eventsday.php", {"d": DAY, "s": "Soccer"}).get("events") or [])
print(f"Events on {DAY}:", len(day_events))
for e in day_events[:10]:
    print("-", e.get("strLeague"), "|", e.get("dateEvent"), e.get("strEvent"), "| id=", e.get("idEvent"))

Events on 2025-08-17: 1
- Australian A-League | 2014-10-10 Melbourne Victory vs Western Sydney Wanderers FC | id= 441043


## Team past/next matches

In [15]:
def team_matches(team_id: str, kind="last", limit=5):
    if kind == "last":
        raw = (GET("eventslast.php", {"id": team_id}).get("results") or [])
    else:
        raw = (GET("eventsnext.php", {"id": team_id}).get("events") or [])
    return raw[:limit]

tm_past = team_matches(TEAM_ID, "last", 5)
tm_next = team_matches(TEAM_ID, "next", 5)
print("Team last:", len(tm_past), "| Team next:", len(tm_next))
for e in tm_past:
    print("-", e.get("dateEvent"), e.get("strEvent"), e.get("intHomeScore"), e.get("intAwayScore"), "| id=", e.get("idEvent"))

Team last: 5 | Team next: 5
- 2025-08-20 Bolton Wanderers vs Reading 1 1 | id= 2274670
- 2025-08-13 Bolton Wanderers vs Sheffield Wednesday 3 3 | id= 2281210
- 2025-08-09 Bolton Wanderers vs Plymouth Argyle 2 0 | id= 2274648
- 2025-07-26 Bolton Wanderers vs Preston North End 2 0 | id= 2294131
- 2025-05-03 Bolton Wanderers vs Stevenage 1 1 | id= 2081485


## Full match package (event summary + timeline + flags)

In [16]:
# choose any event id you saw above
EVENT_ID = (past or tm_past or day_events or [{}])[0].get("idEvent")
print("Using EVENT_ID:", EVENT_ID)

detail = GET("lookupevent.php", {"id": EVENT_ID})
event = (detail.get("events") or [{}])[0]
print("\n=== Event Summary ===")
for k in ["idEvent","dateEvent","strEvent","strLeague","strHomeTeam","strAwayTeam",
          "intHomeScore","intAwayScore","strStatus","strVenue","strVideo","strThumb"]:
    print(f"{k:>12}: {event.get(k)}")

timeline = (GET("lookuptimeline.php", {"id": EVENT_ID}).get("timeline") or [])
print("\n=== Timeline ===")
for t in timeline[:10]:
    m = t.get("intTime")
    etype = t.get("strEvent")
    who = t.get("strPlayer")
    team = t.get("strTeam")
    detail = t.get("strDetail")
    print(f"- {m}′ {etype} — {who} ({team}) {('| ' + detail) if detail else ''}")

stats = (GET("lookupeventstats.php", {"id": EVENT_ID}).get("eventstats") or [])
print("\n=== Stats (first 10) ===")
for s in stats[:10]:
    print("-", s.get("strStat"), ":", s.get("intHome"), "-", s.get("intAway"))

lineup = (GET("lookuplineup.php", {"id": EVENT_ID}).get("lineup") or [])
print("\nHas lineup?", bool(lineup))

Using EVENT_ID: 2274670

=== Event Summary ===
     idEvent: 441613
   dateEvent: 2014-12-29
    strEvent: Liverpool vs Swansea
   strLeague: English Premier League
 strHomeTeam: Liverpool
 strAwayTeam: Swansea
intHomeScore: 4
intAwayScore: 1
   strStatus: None
    strVenue: Anfield
    strVideo: 
    strThumb: None

=== Timeline ===


AttributeError: 'str' object has no attribute 'get'